In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import requests

In [2]:
url = 'https://labdados.com/produtos'
response = requests.get(url)
dados = pd.DataFrame.from_dict(response.json())
dados['Data da Compra'] = pd.to_datetime(dados['Data da Compra'], format = '%d/%m/%Y')

In [3]:
vendas_estados = dados.groupby('Local da compra')[['Preço']].count()
vendas_estados = dados.drop_duplicates(subset = 'Local da compra')[['Local da compra', 'lat', 'lon']].merge(vendas_estados, left_on = 'Local da compra', right_index = True).sort_values('Preço', ascending = False)
vendas_estados.rename(columns = {'Preço': 'Vendas'}, inplace = True)
vendas_estados.head()

,Local da compra,lat,lon,Vendas
1,SP,-22.19,-48.79,3943
2,RJ,-22.25,-42.66,1212
7,MG,-18.10,-44.38,1102
15,RS,-30.17,-53.50,534
16,PR,-24.89,-51.55,473


In [4]:
fig_mapa_vendas = px.scatter_geo(vendas_estados,
                                  lat = 'lat',
                                  lon = 'lon',
                                  scope = 'south america',
                                  size = 'Vendas',
                                  template = 'seaborn',
                                  hover_name = 'Local da compra',
                                  hover_data = {'lat': False, 'lon': False},
                                  title = 'Vendas por estado')
fig_mapa_vendas.show()

In [9]:
vendas_categorias = dados.groupby("Categoria do Produto")[['Preço']].count().sort_values(by = 'Preço', ascending=False).reset_index()
vendas_categorias.rename(columns = {'Preço': 'Vendas'}, inplace = True)
vendas_categorias.head()

,Categoria do Produto,Vendas
0,moveis,1886
1,eletronicos,1772
2,brinquedos,1290
3,eletrodomesticos,1149
4,esporte e lazer,1113


In [6]:
vendas_mensal = dados.set_index('Data da Compra').groupby(pd.Grouper(freq = 'ME'))['Preço'].count().reset_index()
vendas_mensal['Ano'] = vendas_mensal['Data da Compra'].dt.year
vendas_mensal['Mes'] = vendas_mensal['Data da Compra'].dt.month
vendas_mensal.rename(columns = {'Preço': 'Vendas'}, inplace = True)
vendas_mensal.head()

,Data da Compra,Vendas,Ano,Mes
0,2020-01-31,267,2020,1
1,2020-02-29,226,2020,2
2,2020-03-31,284,2020,3
3,2020-04-30,243,2020,4
4,2020-05-31,239,2020,5


In [7]:
fig_vendas_mensal = px.line(
    vendas_mensal,
    x = 'Mes',
    y = 'Vendas',
    markers = True,
    range_y = (0, vendas_mensal.max()),
    color = 'Ano',
    line_dash = 'Ano',
    title = 'Vendas Mensais'
)
fig_vendas_mensal.show()